In [1]:
import torch
import time
import numpy as np
import rdkit

from ase import Atoms
from ase.visualize import view
from ase.build import bulk
from ase.units import GPa
from ase.spacegroup import crystal

from mattersim.applications.batch_relax import BatchRelaxer
from mattersim.forcefield.potential import Potential
from mattersim.datasets.utils.build import build_dataloader
from mattersim.forcefield.potential import MatterSimCalculator
from mattersim.applications.relax import Relaxer

from utils.visualisation import plot_potential, plot_relaxation, visualise_structure

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"MatterSim running on {device}")

MatterSim running on cuda


In [3]:
model = 5

In [4]:
# Define the L1₀ unit cell parameters
a = 3.85  # Lattice constant in x and y
c = 3.72  # Lattice constant in z (slightly compressed)
alpha, beta, gamma = 90, 90, 90

# Define the L1₀ structure using ASE
structure = crystal(
    symbols=['Fe', 'Pt'],
    basis=[[0, 0, 0], [0.5, 0.5, 0.5]],  # Atomic positions as fractional coordinates
    spacegroup=123,  # P4/mmm
    cellpar=[a, a, c, alpha, beta, gamma]  # a, b, c, alpha, beta, gamma
)

In [5]:
_ = visualise_structure(structure)

Structure positions: [[0.    0.    0.   ]
 [1.925 1.925 1.86 ]]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [6]:
structure.calc = MatterSimCalculator(load_path=f"MatterSim-v1.0.0-{model}M.pth", device=device)

2025-02-04 05:03:56.685 | INFO     | mattersim.forcefield.potential:from_checkpoint:891 - Loading the pre-trained mattersim-v1.0.0-5M.pth model
2025-02-04 05:03:57.652 | INFO     | mattersim.forcefield.potential:from_checkpoint:891 - Loading the pre-trained mattersim-v1.0.0-5M.pth model


/usr/local/lib/python3.12/dist-packages/mattersim/forcefield/potential.py:896: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, map_location=

In [7]:
print(f"Energy (eV)                 = {structure.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {structure.get_potential_energy()/len(structure)}")
print(f"Forces of first atom (eV/A) = {structure.get_forces()[0]}")
print(f"Stress[0][0] (x-x) (eV/A^3) = {structure.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (x-x) (GPa)    = {structure.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -10.912311553955078
Energy per atom (eV/atom)   = -5.456155776977539
Forces of first atom (eV/A) = [ 1.0877848e-06  9.3877316e-07 -1.2814999e-06]
Stress[0][0] (x-x) (eV/A^3) = 0.1328575326803123
Stress[0][0] (x-x) (GPa)    = 21.286123275756832


In [8]:
# initialize the relaxation object
relaxer = Relaxer(
    optimizer="BFGS", # the optimization method
    filter = "FrechetCellFilter",
    # filter="ExpCellFilter"
    # filter = None, # filter to apply to the cell
    constrain_symmetry=True, # whether to constrain the symmetry
)

relaxed_structure = relaxer.relax(structure, steps=500)

      Step     Time          Energy          fmax
BFGS:    0 05:03:57      -10.912312        3.662862
BFGS:    1 05:03:57      -11.506659        4.003235
BFGS:    2 05:03:57      -13.788523        3.172477
BFGS:    3 05:03:57      -14.626232        0.188771
BFGS:    4 05:03:57      -14.628473        0.195118
BFGS:    5 05:03:57      -14.189611        2.630441
BFGS:    6 05:03:57      -14.635092        0.220789
BFGS:    7 05:03:57      -14.641842        0.307623
BFGS:    8 05:03:57      -14.381280        2.273954
BFGS:    9 05:03:57      -14.655121        0.487816
BFGS:   10 05:03:57      -14.663462        0.613782
BFGS:   11 05:03:57      -14.667638        0.853680
BFGS:   12 05:03:57      -14.680410        0.721856
BFGS:   13 05:03:57      -14.690160        0.622377
BFGS:   14 05:03:57      -14.708537        0.355899
BFGS:   15 05:03:57      -14.714394        0.097554
BFGS:   16 05:03:57      -14.714851        0.013351
BFGS:   17 05:03:57      -14.714862        0.001910


In [9]:
structure.get_positions()

array([[0.        , 0.        , 0.        ],
       [1.62015491, 1.62015491, 1.37549654]])

In [10]:
_ = visualise_structure(structure)

Structure positions: [[0.         0.         0.        ]
 [1.62015491 1.62015491 1.37549654]]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [11]:
# Define the L1₀ unit cell parameters
a = 3.85  # Lattice constant in x and y
c = 3.72  # Lattice constant in z (slightly compressed)

# Define the atomic positions and symbols
positions = [
    (0, 0, 0),               # Fe atom
    (0.5 * a, 0.5 * a, 0.5 * c),  # Pt atom
]

symbols = ['Fe', 'Pt']

# Create the unit cell for L1₀ FePt
structure = Atoms(symbols="FePt",
                 positions=positions,
                 cell=[(a, 0, 0), (0, a, 0), (0, 0, c)],
                 pbc=True)  # Periodic boundary conditions

structure.rattle(stdev=0.01)

In [12]:
# # Define the L1₀ unit cell parameters
# a = 3.85  # Lattice constant in x and y
# c = 3.72  # Lattice constant in z (slightly compressed)
# alpha, beta, gamma = 90, 90, 90


# # Define the L1₀ structure using spacegroup
# structure = crystal(
#     symbols=['Fe', 'Pt'],
#     basis=[[0, 0, 0], [0.5, 0.5, 0.5]],  # Atomic positions as fractional coordinates
#     spacegroup=123,  # P4/mmm
#     cellpar=[a, a, c, alpha, beta, gamma]  # a, b, c, alpha, beta, gamma
# )

# structure.rattle(stdev=0.01)

In [13]:
structure.get_scaled_positions()

array([[0.00129017, 0.99964087, 0.0017411 ],
       [0.50395592, 0.49939181, 0.4993706 ]])

In [14]:
_ = visualise_structure(structure)

Structure positions: [[ 4.96714153e-03 -1.38264301e-03  6.47688538e-03]
 [ 1.94023030e+00  1.92265847e+00  1.85765863e+00]]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [15]:
structure.calc = MatterSimCalculator(load_path=f"MatterSim-v1.0.0-{model}M.pth", device=device)

/usr/local/lib/python3.12/dist-packages/mattersim/forcefield/potential.py:896: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, map_location=

In [16]:
print(f"Energy (eV)                 = {structure.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {structure.get_potential_energy()/len(structure)}")
print(f"Forces of first atom (eV/A) = {structure.get_forces()[0]}")
print(f"Stress[0][0] (x-x) (eV/A^3) = {structure.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (x-x) (GPa)    = {structure.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -10.912513732910156
Energy per atom (eV/atom)   = -5.456256866455078
Forces of first atom (eV/A) = [-0.02334106  0.0021806   0.0181929 ]
Stress[0][0] (x-x) (eV/A^3) = 0.1328592945809292
Stress[0][0] (x-x) (GPa)    = 21.286405563354492


In [17]:
# initialize the relaxation object
relaxer = Relaxer(
    optimizer="BFGS", # the optimization method
    filter = "FrechetCellfilter",
    # filter="ExpCellFilter", # filter to apply to the cell
    constrain_symmetry=True, # whether to constrain the symmetry
)

relaxed_structure = relaxer.relax(structure, fmax=0.001, steps=500)

      Step     Time          Energy          fmax
BFGS:    0 05:03:57      -10.912512        3.663199
BFGS:    1 05:03:57      -11.506921        4.003320
BFGS:    2 05:03:57      -13.788439        3.171814
BFGS:    3 05:03:57      -14.625860        0.190155
BFGS:    4 05:03:57      -14.628172        0.196123
BFGS:    5 05:03:57      -14.184110        2.640508
BFGS:    6 05:03:57      -14.634958        0.221207
BFGS:    7 05:03:57      -14.641796        0.308700
BFGS:    8 05:03:57      -14.385566        2.259005
BFGS:    9 05:03:57      -14.655932        0.493695
BFGS:   10 05:03:57      -14.665010        0.618661
BFGS:   11 05:03:57      -14.667042        0.805320
BFGS:   12 05:03:57      -14.683666        0.695365
BFGS:   13 05:03:58      -14.692745        0.599825
BFGS:   14 05:03:58      -14.707539        0.337819
BFGS:   15 05:03:58      -14.714073        0.093157
BFGS:   16 05:03:58      -14.714493        0.040560
BFGS:   17 05:03:58      -14.714563        0.037351
BFGS:   18 05:

In [18]:
structure.get_positions()

array([[ 7.14647406e-03, -1.35004234e-03,  1.53371812e-03],
       [ 1.38164908e+00,  1.87826131e+00,  1.37351683e+00]])

In [19]:
print(f"Energy (eV)                 = {structure.get_potential_energy()}")
print(f"Energy per atom (eV/atom)   = {structure.get_potential_energy()/len(structure)}")
print(f"Forces of first atom (eV/A) = {structure.get_forces()[0]}")
print(f"Stress[0][0] (x-x) (eV/A^3) = {structure.get_stress(voigt=False)[0][0]}")
print(f"Stress[0][0] (x-x) (GPa)    = {structure.get_stress(voigt=False)[0][0] / GPa}")

Energy (eV)                 = -14.96288776397705
Energy per atom (eV/atom)   = -7.481443881988525
Forces of first atom (eV/A) = [ 3.5349838e-04 -2.8871000e-08 -7.5320480e-05]
Stress[0][0] (x-x) (eV/A^3) = -6.1466283153366575e-06
Stress[0][0] (x-x) (GPa)    = -0.0009847984183579683


In [20]:
_ = visualise_structure(structure)

Structure positions: [[ 7.14647406e-03 -1.35004234e-03  1.53371812e-03]
 [ 1.38164908e+00  1.87826131e+00  1.37351683e+00]]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [21]:
structure.cell.cellpar()

array([ 2.74629392,  3.75922271,  2.74642348, 90.        , 89.99856751,
       90.        ])